In [1]:
import pandas as pd
from simpledbf import Dbf5
import numpy as np
from tqdm import tqdm
import time
pd.set_option('display.max_columns', 500)

In [2]:
post = pd.read_csv('Post_GPS.csv')

In [4]:
post.head()

,Unnamed: 0,Class,Post_NM,ADDR,ADDR_Road,경도,위도
0,0,서울지방우정청,서울중앙우체국,서울특별시 중구 충무로1가 21-1,서울특별시 중구 소공로 70,126.982115,37.561563
1,1,서울지방우정청,서울을지로4가우체국,서울특별시 중구 을지로4가 312-1,서울특별시 중구 을지로 154-2,126.995176,37.566199
2,2,서울지방우정청,서울태평로우체국,서울특별시 중구 봉래동1가 5-36 우리빌딩 2층,서울특별시 중구 칠패로 42,126.973586,37.559449
3,3,서울지방우정청,서울퇴계로5가우체국,서울특별시 중구 장충동2가 173-3,서울특별시 중구 퇴계로 270,127.001668,37.562745
4,4,서울지방우정청,서울역전우체국,서울특별시 중구 봉래동2가 123,서울특별시 중구 통일로 21,126.971413,37.557587


In [5]:
dbf = Dbf5('Public.dbf',codec='euc-kr')
public = dbf.to_dataframe()
public.head()

,유형,상세분류,시군구코드,도로명코드,도로명주소,기관명,위치X,위치Y,전화번호
0,경찰서,지방청,11110,4100135,서울특별시 종로구 사직로8길 31 (내자동),서울특별시지방경찰청,9.533747e+05,1.952964e+06,182
1,경찰서,지방청,26470,2000010,부산광역시 연제구 중앙대로 999 (연산동),부산광역시지방경찰청,1.143370e+06,1.688228e+06,182
2,경찰서,지방청,27230,3145034,대구광역시 북구 연암로 40 (산격동),경상북도지방경찰청,1.099134e+06,1.766715e+06,182
3,경찰서,지방청,27260,3146011,대구광역시 수성구 무학로 227 (지산동),대구광역시지방경찰청,1.102491e+06,1.759815e+06,182
4,경찰서,지방청,28200,4259535,인천광역시 남동구 예술로152번길 9 (구월동),인천광역시지방경찰청,9.293508e+05,1.939185e+06,182


In [27]:
# !pip install pyproj
from pyproj import Proj,transform
proj_UTMK=Proj(init='epsg:5178')
proj_WGS84=Proj(init='epsg:4326')

/opt/anaconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/anaconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [28]:
def transform_utmk_to_w84(df):
    return pd.Series(transform(proj_UTMK, proj_WGS84, df['위치X'],df['위치Y']),index=['위치X','위치Y'])

In [29]:
public[['경도','위도']] = public.apply(transform_utmk_to_w84,axis=1)

/var/folders/_w/87c9vj2108j57km3_dpjq_kr0000gn/T/ipykernel_1013/244005634.py:2: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return pd.Series(transform(proj_UTMK, proj_WGS84, df['위치X'],df['위치Y']),index=['위치X','위치Y'])


In [30]:
public.head()

,유형,상세분류,시군구코드,도로명코드,도로명주소,기관명,위치X,위치Y,전화번호,경도,위도
0,경찰서,지방청,11110,4100135,서울특별시 종로구 사직로8길 31 (내자동),서울특별시지방경찰청,9.533747e+05,1.952964e+06,182,126.969817,37.577625
1,경찰서,지방청,26470,2000010,부산광역시 연제구 중앙대로 999 (연산동),부산광역시지방경찰청,1.143370e+06,1.688228e+06,182,129.072438,35.181909
2,경찰서,지방청,27230,3145034,대구광역시 북구 연암로 40 (산격동),경상북도지방경찰청,1.099134e+06,1.766715e+06,182,128.596310,35.894800
3,경찰서,지방청,27260,3146011,대구광역시 수성구 무학로 227 (지산동),대구광역시지방경찰청,1.102491e+06,1.759815e+06,182,128.632617,35.832257
4,경찰서,지방청,28200,4259535,인천광역시 남동구 예술로152번길 9 (구월동),인천광역시지방경찰청,9.293508e+05,1.939185e+06,182,126.699090,37.451902


In [31]:
public.to_csv('Public.csv')

In [79]:
public = pd.read_csv('public.csv')

In [36]:
import numbers
import math

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)

In [80]:
post['경도']=pd.to_numeric(post['경도'])
post['위도']=pd.to_numeric(post['위도'])
public['경도']=pd.to_numeric(public['경도'])
public['위도']=pd.to_numeric(public['위도'])

In [81]:
# 포함하고자 하는 문자열 리스트 생성
example_list = ['학교', '자치단체','경찰서','국세']

# join함수를 이용하여 이어주고 contains 함수에 넣기
test = '|'.join(example_list)
public = public[public['유형'].str.contains(test)]

In [82]:
public=public.dropna(subset=['위도'])
public=public.dropna(subset=['경도'])
public = public.reset_index(drop=True)
public = public.replace([np.inf, -np.inf], None)

In [83]:
post=post.dropna(subset=['위도'])
post=post.dropna(subset=['경도'])
post = post.reset_index(drop=True)
post = post.replace([np.inf, -np.inf], None)

In [84]:
result=[]
result.append([])
for i in tqdm(post.index):
    sub_lat =  post.loc[i,'위도']
    sub_long = post.loc[i,'경도']
    result[i].append(post.loc[i,'Post_NM'])
    for j in public.index:
        lat =  public.loc[j,'위도']
        long = public.loc[j,'경도']
        temp = GeoUtil.get_harversion_distance(sub_long, sub_lat,long, lat)
        if temp <1:
            result[i].append(public.loc[j,'기관명'])
    result.append([])

100%|██████████████████████████████████████████████████████████████████████████████████████████| 3309/3309 [22:17<00:00,  2.47it/s]


In [86]:
df=pd.DataFrame(result)

In [89]:
df.to_csv('result.csv')